# CHI Square Test
To further look into differences in our data, we conduct multiple chi² tests to see if there are any significant differences between females/males and grad students/PhDs regarding different emotion, affect, level of interest and arousal valence attributes.

## Import relevant libraries

In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import sklearn.preprocessing as pp
import statsmodels.api as sm
#import nltk
import scipy.stats as st
import statsmodels.formula.api as smf
import seaborn as sns
import Helper as hp

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


## Load .csv data with results of OpenSMILE Analysis
First we load .csv data and clean it (removing of NaNs), then we store information of all files in seperate panda dataframes containing information about affect, emotion and valence/arousal for all participants.

In [2]:
data = pd.read_csv("CHI_2019_FULL.csv")

#Set Labels 
emotion_label = ['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness']
affect_label = ['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired']
loi_label = ['Disinterest', 'Normal', 'High Interest']

#Get specific data and save it into new data frames
# We use the pandas .copy(deep=True) function to prevent the SettingWithCopyWarning we would otherwise get. Since we do
# not write, but only read from the data, the warning does not affect the data frames
df_emotion = data[['Anger', 'Boredom', 'Disgust', 'Fear', 'Happiness', 'Emo_Neutral', 'Sadness', 'Filename']].copy(deep=True)
df_affect = data[['Aggressiv', 'Cheerful', 'Intoxicated', 'Nervous', 'Aff_Neutral', 'Tired', 'Filename']].copy(deep=True)
df_loi = data[['Disinterest', 'Normal', 'High Interest', 'Filename']].copy(deep=True)
df_ar_val = data[['Arousal', 'Valence', 'Filename']].copy(deep=True)
#For further usage, we want to append the CharacterID as a column, which is saved with other information in the filename
#Since we only want the digits, we can remove all non-digit characters of the filename column and append the column to the df

df_emotion['Char_ID'] = df_emotion['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_affect['Char_ID'] = df_affect['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_loi['Char_ID'] = df_loi['Filename'].replace('\D+','', regex = True).copy(deep=True)
df_ar_val['Char_ID'] = df_ar_val['Filename'].replace('\D+','', regex = True).copy(deep=True)



## Let's load information about the speakers
The speaker ID is saved in a single .csv file containing four important columns: ID, Age, Sex and Acadedmic Status. Since before loaded OpenSMILE csv files are named using the corresponding index (ex. speaker with id 0 has two files 0_a.csv and 0_b.csv), so that a link can be created

In [3]:
char_data = pd.read_csv("CHI_2019_CharacterData.csv")  

#Join above tables and Character Tables

#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['ID'] = char_data['ID'].astype(int)
df_ar_val['Char_ID'] = df_ar_val['Char_ID'].astype(int)
df_emotion['Char_ID'] = df_emotion['Char_ID'].astype(int)
df_affect['Char_ID'] = df_affect['Char_ID'].astype(int)
df_loi['Char_ID'] = df_loi['Char_ID'].astype(int)

#Safe new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='Char_ID', right_on='ID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='Char_ID', right_on= 'ID')

#Now, we only want to have data containing information about the answers
#For that we need to extract from the filename column, whether the file was part of an answer
#a = answer, p = presentation, q = question
#sentence_type should be the same for all tables, but just to be sure
arval_sentence_type = df_ar_val_char.Filename.str.replace('\d+','').str[3:-4]
df_ar_val_char['SentenceType'] = arval_sentence_type
emo_sentence_type = df_emotion_char.Filename.str.replace('\d+','').str[3:-4]
df_emotion_char['SentenceType'] = emo_sentence_type
aff_sentence_type = df_affect_char.Filename.str.replace('\d+','').str[3:-4]
df_affect_char['SentenceType'] = aff_sentence_type
loi_sentence_type = df_loi_char.Filename.str.replace('\d+','').str[3:-4]
df_loi_char['SentenceType'] = loi_sentence_type

#Now select only those who have SentenceType == 'a'
df_ar_val_char = df_ar_val_char.loc[df_ar_val_char['SentenceType'] == 'p']
df_emotion_char = df_emotion_char.loc[df_emotion_char['SentenceType'] == 'p']
df_affect_char = df_affect_char.loc[df_affect_char['SentenceType'] == 'p']
df_loi_char = df_loi_char.loc[df_loi_char['SentenceType'] == 'p']

#For affect, we will have to drop the intoxication column and thus we will re-normalize the other values
affect_label.remove('Intoxicated')
df_affect_char = df_affect_char.drop(['Intoxicated'], axis = 1)
norm_test = pp.normalize(df_affect_char[affect_label], norm = 'l1')
df_affect_char[affect_label] = norm_test

df_loi_char['Normal Interest'] = df_loi_char['Disinterest'] + df_loi_char['Normal']
df_loi_char = df_loi_char.drop(['Disinterest', 'Normal'], axis = 1)
loi_label = ['Normal Interest', 'High Interest']

#Now let's compensate for multiple samples of the same person
IDs = df_emotion_char['Char_ID'].copy()
IDs.drop_duplicates()

# I know that for loops make the following operations very very slow, but I could not find a way to get this
# working with pd.apply and lambda functions... 

for i in IDs:
    #first for emotion
    for l in emotion_label:    
        emo = df_emotion_char.loc[df_emotion_char.Char_ID == i,l]
        emo_neu = hp.constructMedianSeries(emo)
        df_emotion_char.loc[df_emotion_char.Char_ID == i,l] = emo_neu
    #Then for affect
    for l in affect_label:    
        aff = df_affect_char.loc[df_affect_char.Char_ID == i,l]
        aff_neu = hp.constructMedianSeries(aff)
        df_affect_char.loc[df_affect_char.Char_ID == i,l] = aff_neu  
    for l in loi_label:    
        loi = df_loi_char.loc[df_loi_char.Char_ID == i,l]
        loi_neu = hp.constructMedianSeries(loi)
        df_loi_char.loc[df_loi_char.Char_ID == i,l] = loi_neu  
    for l in ['Arousal', 'Valence']:    
        arval = df_ar_val_char.loc[df_ar_val_char.Char_ID == i,l]
        arval_neu = hp.constructMedianSeries(arval)
        df_ar_val_char.loc[df_ar_val_char.Char_ID == i,l] = arval_neu  
        
#Now let's drop the duplicate values, since we only need one row now per person
df_emotion_char.drop_duplicates(subset=['ID'], inplace = True)
df_affect_char.drop_duplicates(subset=['ID'], inplace = True)
df_loi_char.drop_duplicates(subset=['ID'], inplace = True)
df_ar_val_char.drop_duplicates(subset=['ID'], inplace = True)    

## Chi-squared Test of Independence
We Start with characteristic sex. The null hypothesis states that the two categorical variables sex and e.g. emotion are independent.

Since we have float data and chi² needs integer data, such as observation counts, we have to convert our data. To illustrate how this is done, we'll look at a specific emotion, 'Anger'. We need to make sure that in our observation count, we do not have any cells with a value of less than 5, since this yields errors and may falsify the result. So we calculate the quantiles of our emotion 'Anger', which yields us three thresholds to compare the float data. This way, we can count how many samples were in the 1st, 2nd, 3rd or 4th Quantile. We want to compare two (or more) groups, so we compare only the female values and sort them into quartiles, then for male values. This yields a 2x4 table. An example table is printed below. This table is used to calculate the chi2 statistic. Note that the function 'calcFrequencyTable' takes in a pd.DataFrame, not a pd.Series and returns an array of pd.DataFrames. This means, that the function calculates these tables for all different emotions defined in e.g. emotion_label.

In [4]:
#Example Frequency Table for the emotion 'Anger':
#Since the function does the table calculation for all different emotions, we only want to select the first table
#which holds the table for 'anger' (since it's the first element, see declaration of emotion_label at the start)
anger_table = hp.calcFrequencyTable(df_emotion_char, emotion_label, 'Sex')[0]
anger_table

,1st Quartile,2nd Quartile,3rd Quartile,4th Quartile
Male,10,11,9,10
Female,11,9,11,10


In [5]:
print('EMOTION\n')
emo_sex_chi2 = hp.chi2(df_emotion_char, emotion_label,'Sex',  True)
print('\nAFFECT\n')
aff_sec_chi2 = hp.chi2(df_affect_char, affect_label,'Sex',  True)
print('\nAROUSAL-VALENCE\n')
ar_val_sec_chi2 = hp.chi2(df_ar_val_char, ['Arousal', 'Valence'], 'Sex', True)
print('\nLEVEL OF INTEREST\n')
loi_sec_chi2 = hp.chi2(df_loi_char, ['Normal Interest', 'High Interest'], 'Sex', True)
print('\nResiduals of: '+ affect_label[4])
aff_sec_chi2[1][4]

EMOTION

Chi square of Anger : 0.43533972125435544 with p-value of: 0.9328491649308727
Cramers V: 0.07331142107427951
Chi square of Boredom : 1.9784320557491293 with p-value of: 0.5768952587117124
Cramers V: 0.15628527476385592
Chi square of Disgust : 20.4479137630662 with p-value of: 0.0001370628588903404
Cramers V: 0.5024374386836044
Chi square of Fear : 7.44593205574913 with p-value of: 0.05896408489675078
Cramers V: 0.303191502367751
Chi square of Happiness : 5.178919860627178 with p-value of: 0.15915488787442197
Cramers V: 0.25285822555330323
Chi square of Emo_Neutral : 3.321493902439025 with p-value of: 0.34466228008084693
Cramers V: 0.2024996236069974
Chi square of Sadness : 15.551929442508714 with p-value of: 0.001400876792680575
Cramers V: 0.43817705202333523

AFFECT

Chi square of Aggressiv : 8.046023519163763 with p-value of: 0.0450700529351656
Cramers V: 0.3151723713991754
Chi square of Cheerful : 1.416441637630662 with p-value of: 0.7016852861743752
Cramers V: 0.1322381707

,1st Quartile,2nd Quartile,3rd Quartile,4th Quartile
Male,0.319305,-1.997884,0.063627,1.60976
Female,-0.319305,1.997884,-0.063627,-1.60976


If we take a look at the above p-values, we see significant differences in emotion for disgust, fear and sadness. Regarding the residuals, we see that females are more disgusted than males (4th quartile ~6.25), 1st and 2nd quartile for males: ~4.70 and ~2.03. Further looking at fear, we see that females tend to have more 'extreme' values, meaning that the 1st quartile and 4th quartile are positive (~2.11 and ~0.92). Looking at sadness, we see that (just as in the previous analysis) males tend to be more sad than females (3rd quartile ~1.3, 4th quartile ~4.70).

Looking at the p-values of affect, we can see significant difference for aggressive, cheerful, intoxicated and tired. Further analyzing the residuals, we can see that females tend to be more aggressive (4th quartile ~4.18) than males (1st quartile ~3.51, 2nd quartile ~1.44). Looking at cheerful we see that males tend to have higher values (4th quartile ~3.51) than females (1st to 3rd quartile positive). Regarding intoxication, we see that females tend to have values above the median (3rd and 4th quartile positive (~1.68 and ~2.70), while males tend to have values below the median (1st quartile ~2.92, 2nd quartile ~1.44). Lastly looking at tiredness, we see that males tend to be more tired than females, as males have positive values for the 3rd and 4th quartile, and females have positive values for the 1st and 2nd quartile.

If we take a look at the p-values for arousal valence, we see arousal and valence show significant differences between males and females. Looking at the residuals, we can see that females have higher values of arousal than males (1st quartile for males ~6.18, 3rd Q for females ~2.27 and 4th Q for females ~3.59). Regarding valence, we see that females tend to have values below the median (positive values for 1st Q ~3.88 and 2nd Q ~1.22), whereas males tend to have values above the median (3rd Q ~3.68; 4th Q ~1.44).

Lastly if we take a look at the p-values for level of interest, we can see significant differences for disinterest, normal and high interest. Further investigating the residuals tells us, that females tend to have lower values for disinterest than males (1st quartile ~3.88 for females). Furthermore females tend to have lower values for normal interest than males (1st quartile ~3.29 for females). This implies, that females will have higher values for high interest. By looking at the residuals we are able to confirm this: the only positive value for females is at the 4th quartile (~3.29).

Now move on to academic status, the hypothesis being that the variables academic status and e.g. emotion are independent.

In [6]:
print('EMOTION\n')
emo_aca_chi2 = hp.chi2(df_emotion_char, emotion_label,'Academic' , True)
print('\nAFFECT\n')
aff_aca_chi2 = hp.chi2(df_affect_char, affect_label,'Academic', True)
print('\nAROUSAL-VALENCE\n')
ar_val_aca_chi2 = hp.chi2(df_ar_val_char, ['Arousal', 'Valence'],  'Academic',True)
print('\nLEVEL OF INTEREST\n')
loi_aca_chi2 = hp.chi2(df_loi_char, ['Normal Interest', 'High Interest'],'Academic', True)
print('\nResiduals of: '+ affect_label[2])
ar_val_aca_chi2[1][1]

EMOTION

Chi square of Anger : 2.9518449030644147 with p-value of: 0.39910949093209325
Cramers V: 0.19089926575935778
Chi square of Boredom : 6.790334184429176 with p-value of: 0.07888943682072329
Cramers V: 0.28953632971964105
Chi square of Disgust : 6.554096310193872 with p-value of: 0.0875533198777855
Cramers V: 0.2844552148610616
Chi square of Fear : 5.353345841150719 with p-value of: 0.1476780745346126
Cramers V: 0.25708109498156895
Chi square of Happiness : 4.953095684803002 with p-value of: 0.17526410464554013
Cramers V: 0.2472839045348734
Chi square of Emo_Neutral : 6.790334184429176 with p-value of: 0.07888943682072329
Cramers V: 0.28953632971964105
Chi square of Sadness : 1.4320980688118046 with p-value of: 0.698028313491549
Cramers V: 0.1329669999351368

AFFECT

Chi square of Aggressiv : 4.152595372107567 with p-value of: 0.24545104939706014
Cramers V: 0.22642130980142347
Chi square of Cheerful : 3.75234521575985 with p-value of: 0.28947805322214165
Cramers V: 0.215233012285

,1st Quartile,2nd Quartile,3rd Quartile,4th Quartile
Grad Student,2.154194,-0.913259,-0.12914,-1.162258
PhD,-2.154194,0.913259,0.12914,1.162258


Looking at the p-values for emotion, we see significant differences between Grad Students and PhDs for boredom and neutral (emotion). By further looking at the residuals, we are able to see that PhDs tend to have lower values (1st quartile ~3.21) for boredom than grad students (2nd quartile ~2.76). Looking at neutral (emotion), we see that PhDs tend to be more 'extreme', meaning a value of ~3.21 for the 1st quartile and a value of ~1.42 for the 4th quartile.

If we take a look at the p-values for affect, we only see statistical significant differences in intoxication. Looking at the residuals we see that grad students tend to have lower values for intoxication (1st Q ~2.03) but also have a value of ~0.52 for the 4th quartile. But we can say that the main differences in intoxication between gard students and phds lies between the 1st and 3rd quartile, where phds tend to be more intoxicated (3rd Q ~2.47).

Regarding the p-values of arousal-valence, we can only see a statistical significant difference between PhDs and grad students in valence. Looking at the residuals we can say that grad students (1st Q ~2.46, 3rd ~0.53) tend to have smaller valence values than PhDs (2nd Q ~1.26, 4th Q ~1.71)

Now let's look at Native Speaker.

In [7]:
print('EMOTION\n')
emo_age_chi2 = hp.chi2(df_emotion_char, emotion_label,'IsNativeSpeaker', True)
print('\nAFFECT\n')
aff_age_chi2 = hp.chi2(df_affect_char, affect_label, 'IsNativeSpeaker', True)
print('\nAROUSAL-VALENCE\n')
ar_val_age_chi2 = hp.chi2(df_ar_val_char, ['Arousal', 'Valence'],'IsNativeSpeaker' ,True)
print('\nLEVEL OF INTEREST\n')
loi_age_chi2 = hp.chi2(df_loi_char, ['Normal Interest', 'High Interest'], 'IsNativeSpeaker',  True)

EMOTION

Chi square of Anger : 2.600965250965251 with p-value of: 0.8570013311381182
Cramers V: 0.17919453705591057
Chi square of Boredom : 8.587118287118287 with p-value of: 0.19816403314894113
Cramers V: 0.3255976136488196
Chi square of Disgust : 8.490926640926642 with p-value of: 0.20429622527018043
Cramers V: 0.3237688292999418
Chi square of Fear : 6.569936819936822 with p-value of: 0.3624541777587051
Cramers V: 0.284798755457834
Chi square of Happiness : 8.061618111618113 with p-value of: 0.23362368561135471
Cramers V: 0.31547765297426306
Chi square of Emo_Neutral : 6.667786942786944 with p-value of: 0.3526651557515303
Cramers V: 0.2869117587662753
Chi square of Sadness : 7.884161109161108 with p-value of: 0.24671342862313786
Cramers V: 0.311986093176158

AFFECT

Chi square of Aggressiv : 15.02864162864163 with p-value of: 0.02003511358751028
Cramers V: 0.4307421334612921
Chi square of Cheerful : 2.5057213057213055 with p-value of: 0.8678268114920646
Cramers V: 0.17588300354164677

## Post-Hoc tests for age and native speaker, as they have three different groups

If a significant p-value for the category 'NativeSpeaker' is found, we do not yet know which groups differ significantly from each other, so post-hoc testing is done for this character feature.

In [8]:
print('EMOTION\n')
print('post-hoc emotions and different groups')
emo_reject_list, emo_corrected_p_vals, emo_combinations, emo_residuals= hp.chi2_post_hoc(df_emotion_char,emotion_label, 'IsNativeSpeaker', 'bonferroni', True, True)
print('\nAFFECT\n')
print('\n post-hoc affect and different groups')
aff_reject_list, emo_corrected_p_vals, emo_combinations, aff_residuals = hp.chi2_post_hoc(df_affect_char, affect_label, 'IsNativeSpeaker' ,'bonferroni', True, True)
print('\nAROUSAL-VALENCE\n')
print('\n post-hoc arousal-valence and different groups')
ar_val_reject_list, ar_val_corrected_p_vals, ar_val_combinations, ar_val_residuals = hp.chi2_post_hoc(df_ar_val_char, ['Arousal', 'Valence'], 'IsNativeSpeaker', 'bonferroni',True, True)
print('\nLEVEL OF INTEREST\n')
print('\n post-hoc level of intereset and different groups')
loi_reject_list, loi_corrected_p_vals, loi_combinations, loi_residuals = hp.chi2_post_hoc(df_loi_char, ['Normal Interest', 'High Interest'], 'IsNativeSpeaker', 'bonferroni', True, True)

EMOTION

post-hoc emotions and different groups
Anger
Combinations: [('Asian Non-Native', 'Europ. Non-Native'), ('Asian Non-Native', 'Native Speaker'), ('Europ. Non-Native', 'Native Speaker')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]
Boredom
Combinations: [('Asian Non-Native', 'Europ. Non-Native'), ('Asian Non-Native', 'Native Speaker'), ('Europ. Non-Native', 'Native Speaker')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]
Disgust
Combinations: [('Asian Non-Native', 'Europ. Non-Native'), ('Asian Non-Native', 'Native Speaker'), ('Europ. Non-Native', 'Native Speaker')]
Reject List: [False False False]
Corrected p-values: [0.94972434 1.         1.        ]
Fear
Combinations: [('Asian Non-Native', 'Europ. Non-Native'), ('Asian Non-Native', 'Native Speaker'), ('Europ. Non-Native', 'Native Speaker')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]
Happiness
Combinations: [('Asian Non-Native', 'Europ. Non-Native'), ('Asian Non-Native', 